In [ ]:
from src.analytics.models.car_make import CarMakeEmbedding
from sklearn.preprocessing import OneHotEncoder
from datetime import date
import pandas as pd

In [ ]:
cme = CarMakeEmbedding(
        label_encoder='/Users/ignasi/Documents/_03_MDS_/_01_ADSDB_/src/embeddings/encoder.pkl',
        pkl_path='/data/models/car_make_embedding/embedding.pkl',
    )

## Analytical sandbox

We read the dataset with the following columns:
'person_id', 'person_age', 'person_sex', 'vehicle_make', 'vehicle_year', 'overall_rating'
From these columns and as we want to analyze the vehicle safety rating, we will delete the information related to the driver, so we will delete 'person_id', 'person_age', 'person_sex'. More specifically, we will try to find a relation between makes + years ~ safety. Looking for a pattern in this variables and computing clustering or regressions. 

In [18]:
df = pd.read_csv(
    "/Users/ignasi/Documents/_03_MDS_/_01_ADSDB_/ui/sample_tables/mvc_safety_rating_by_accidents.csv"
)
df = df[["vehicle_make", "vehicle_year", "overall_rating"]]

In [19]:
df['vehicle_year'].max()

2024.0

## Feature engineering

In [20]:
df["vehicle_age"] = date.today().year - df["vehicle_year"]

In [21]:
df["vehicle_make_embedding"] = df["vehicle_make"].apply(lambda x: cme.execute(x)[0])
make_df = pd.DataFrame(
    df["vehicle_make_embedding"].tolist(),
    columns=[f"make_{i}" for i in range(len(df["vehicle_make_embedding"][0]))],
)
df = pd.concat([df.drop(columns=["vehicle_make_embedding"]), make_df], axis=1)
df.dropna(inplace=True)

In [22]:
df = df[[f"make_{i}" for i in range(10)] + ["vehicle_age", "overall_rating"]]

In [27]:
df['overall_rating'] = df["overall_rating"].astype(int)

In [28]:
df.to_csv('brands.csv', index=False)